In [1]:
from pycocotools.coco import COCO
import os
import json
os.chdir("../")

In [2]:
coco_train = COCO("./data/Annotations/coco/instances_train2014.json")
coco_val = COCO("./data/Annotations/coco/instances_val2014.json")

loading annotations into memory...
Done (t=7.78s)
creating index...
index created!
loading annotations into memory...
Done (t=4.13s)
creating index...
index created!


In [3]:
def get_img_info(coco_val):
    outputs = []
    for i in coco_val.getImgIds():
        img_info = {}
        img_info['img_id'] = i
        img_info['file_name'] = coco_val.loadImgs(i)[0]['file_name']
        img_info['width'] = coco_val.loadImgs(i)[0]['width']
        img_info['height'] = coco_val.loadImgs(i)[0]['height']
        
        ann_ids = coco_val.getAnnIds(imgIds=i, catIds=[], iscrowd=False)
        anns = coco_val.loadAnns(ann_ids)
        
        img_info['objects_count'] = len(anns)
        if not anns:
            img_info.update({
                'max_obj_id': None,
                'max_obj_cat_id': None,
                'max_obj_area': None,
                'max_obj_midpoint': None,
                'max_obj_bbox': None,
                'max_obj_segment_points': None,
                'max_obj_area_portion': None
            })
        else:
            sorted_anns = sorted(anns, key=lambda x: x['area'], reverse=True)
            max_ann = sorted_anns[0]
            
            img_info['max_obj_id'] = max_ann['id']
            img_info['max_obj_cat_id'] = max_ann['category_id']
            img_info['max_obj_area'] = max_ann['area']
            img_info['max_obj_midpoint'] = [max_ann['bbox'][0] + max_ann['bbox'][2] / 2, max_ann['bbox'][1] + max_ann['bbox'][3] / 2]
            img_info['max_obj_bbox'] = max_ann['bbox']
            img_info['max_obj_segment_points'] = max_ann['segmentation']
            img_info['max_obj_area_portion'] = max_ann['area'] / (coco_val.loadImgs(i)[0]['height'] * coco_val.loadImgs(i)[0]['width'])
        
        outputs.append(img_info)
        
    return outputs

In [4]:
outputs = get_img_info(coco_val)
with open ("./data/Annotations/coco/val2014_rmg.json", "w") as f:
    json.dump(outputs, f)

In [5]:
outputs = get_img_info(coco_train)
with open ("./data/Annotations/coco/train2014_rmg.json", "w") as f:
    json.dump(outputs, f)

In [6]:
outputs = get_img_info(coco_train)
outputs[0]

{'img_id': 57870,
 'file_name': 'COCO_train2014_000000057870.jpg',
 'width': 640,
 'height': 480,
 'objects_count': 17,
 'max_obj_id': 2190436,
 'max_obj_cat_id': 62,
 'max_obj_area': 23717.007900000004,
 'max_obj_midpoint': [176.59, 375.745],
 'max_obj_bbox': [90.67, 271.49, 171.84, 208.51],
 'max_obj_segment_points': [[91.48,
   476.07,
   90.67,
   425.33,
   130.76,
   394.24,
   126.67,
   374.6,
   116.85,
   348.41,
   119.31,
   326.32,
   136.49,
   297.68,
   161.86,
   281.31,
   189.68,
   271.49,
   217.5,
   272.31,
   247.78,
   287.86,
   262.51,
   310.77,
   259.24,
   327.95,
   255.96,
   348.41,
   243.69,
   372.14,
   228.96,
   404.87,
   223.23,
   432.7,
   236.32,
   480.0,
   217.5,
   477.7,
   204.41,
   437.61,
   193.77,
   431.06,
   191.32,
   474.43,
   182.32,
   478.52,
   163.49,
   478.52,
   173.31,
   453.97,
   174.13,
   435.15,
   161.86,
   453.97,
   155.31,
   480.0,
   136.49,
   480.0,
   129.94,
   473.61,
   127.49,
   464.61,
   121.7